In [1]:
pip install transformers torch pillow

In [4]:
pip install inquirer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 3.6 MB/s eta 0:00:00


In [7]:
import os
from PIL import Image
import torch
from transformers import AutoImageProcessor, SiglipForImageClassification

def load_model(model_name="Ateeqq/ai-vs-human-image-detector", device=None):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Loading model: {model_name} on {device}")
    processor = AutoImageProcessor.from_pretrained(model_name)
    model = SiglipForImageClassification.from_pretrained(model_name)
    model.to(device)
    model.eval()
    return processor, model, device

def predict_image(image_path, processor, model, device):
    # open image
    try:
        img = Image.open(image_path).convert("RGB")
    except Exception as e:
        print(f"Error opening image: {e}")
        return

    # preprocess
    inputs = processor(images=img, return_tensors="pt").to(device)

    # inference
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits  # shape: [1, num_labels]

    # get predicted label
    probs = torch.softmax(logits, dim=-1)
    pred_idx = torch.argmax(logits, dim=-1).item()
    pred_label = model.config.id2label[pred_idx]
    confidence = probs[0, pred_idx].item()

    return pred_label, confidence, probs.squeeze().tolist()

def main():
    processor, model, device = load_model()

    image_path = input("Enter path to image file: ").strip()
    if not os.path.exists(image_path):
        print("File does not exist.")
        return

    label, conf, all_probs = predict_image(image_path, processor, model, device)
    print("=== Result ===")
    print(f"Prediction: {label}")
    print(f"Confidence: {conf:.4f}")
    print("All class probabilities:")
    for idx, prob in enumerate(all_probs):
        lbl = model.config.id2label[idx]
        print(f"  {lbl}: {prob:.4f}")

if __name__ == "__main__":
    main()

Loading model: Ateeqq/ai-vs-human-image-detector on cpu


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Enter path to image file: /content/man.png
=== Result ===
Prediction: ai
Confidence: 1.0000
All class probabilities:
  ai: 1.0000
  hum: 0.0000
